###  Predicting California Housing Prices using all the features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestRegressor

##### Data Ingestion 

In [2]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
df = pd.read_csv(url)
df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Feature Engineering

In [4]:
subset = df.loc[df['ocean_proximity'].isin(['<1H OCEAN','INLAND'])]
subset.reset_index(drop=True)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0,<1H OCEAN
1,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0,<1H OCEAN
2,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0,<1H OCEAN
3,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0,<1H OCEAN
4,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
15682,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
15683,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
15684,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
15685,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


### Preparing the dataset
###### Preparation:

######  Fill missing values with zeros.
######  Apply the log transform to median_house_value.
######  Do train/validation/test split with 60%/20%/20% distribution.
######  Use the train_test_split function and set the random_state parameter to 1.
######  Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [5]:
#check for duplicated values
subset.duplicated().sum()

0

In [6]:
#check for null values
subset.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [7]:
#fill the null value with zero
subset = subset.fillna(0)

In [8]:
subset.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [9]:
#apply log1p transformation to median_house_value
subset   

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0,<1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0,<1H OCEAN
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0,<1H OCEAN
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [10]:
subset.median_house_value = np.log1p(subset.median_house_value.values)
subset.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,12.973866,<1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,12.287657,<1H OCEAN


In [11]:
#split the data into 60%-20%-20%
SEED = 1
df_full_train, df_test = train_test_split(subset, test_size=0.2, random_state=SEED) 
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=SEED)
len(subset), len(df_train),len(df_test),len(df_val), len(df_train+df_val+df_test)

(15687, 9411, 3138, 3138, 15687)

In [12]:
# separate the predicted values / responses
y_train = df_train.median_house_value
y_val = df_val.median_house_value
y_test = df_test.median_house_value

In [13]:
#remove the median_house_value values
del(df_train['median_house_value'])
del(df_val['median_house_value'])
del(df_test['median_house_value'])

In [14]:
# use DictVectorizer(sparse=True) to turn the dataframes into matrices.
train_dict = df_train.to_dict(orient='records')
val_dict = df_val.to_dict(orient='records')
full_dict = df_full_train.to_dict(orient='records')

In [15]:
dv = DictVectorizer(sparse=True)

In [16]:
X_train = dv.fit_transform(train_dict)
#X_val = dv.fit_transform(val_dict)
#X_full = dv.fit_transform(full_dict)

#### Question 1
###### Let's train a decision tree regressor to predict the median_house_value variable.

###### Train a model with max_depth=1.

In [65]:
dtr = DecisionTreeRegressor(max_depth=1)

In [66]:
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [67]:
print(export_text(dtr,feature_names=dv.get_feature_names_out()))

|--- ocean_proximity=<1H OCEAN <= 0.50
|   |--- value: [11.61]
|--- ocean_proximity=<1H OCEAN >  0.50
|   |--- value: [12.30]



#### Question 2
###### Train a random forest model with these parameters:

###### n_estimators=10
###### random_state=1
###### n_jobs=-1 (optional - to make training faster)
###### What's the RMSE of this model on validation?

In [68]:
rfr = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)

In [69]:
rfr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [70]:
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [71]:
y_pred = rfr.predict(X_val)
y_pred

array([12.24038309, 11.24738854, 11.73003605, ..., 11.73063011,
       11.33572008, 11.7964171 ])

In [72]:
score = mean_squared_error(y_test, y_pred, squared=False)
score.round(3)

0.703

#### Question 3

#### Now let's experiment with the n_estimators parameter
###### Try different values of this parameter from 10 to 200 with step 10.
###### Set random_state to 1.
###### Evaluate the model on the validation dataset.
###### After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for retrieving the answer.

In [73]:
def evaluate_rmse(n, max_depth=None, random_state=1):
    
    rfr = RandomForestRegressor(n_estimators=n, random_state=random_state, n_jobs=-1, max_depth=max_depth)
    rfr.fit(X_train, y_train)
    
    val_dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    y_pred = rfr.predict(X_val)
    
    score = mean_squared_error(y_test, y_pred, squared=False).round(3)
    return score

In [74]:
for n in range(10, 210, 10):
    rmse = evaluate_rmse(n)
    print(f"At n: {n} the rmse: {rmse}")

At n: 10 the rmse: 0.703
At n: 20 the rmse: 0.685
At n: 30 the rmse: 0.686
At n: 40 the rmse: 0.685
At n: 50 the rmse: 0.684
At n: 60 the rmse: 0.683
At n: 70 the rmse: 0.685
At n: 80 the rmse: 0.686
At n: 90 the rmse: 0.685
At n: 100 the rmse: 0.684
At n: 110 the rmse: 0.683
At n: 120 the rmse: 0.681
At n: 130 the rmse: 0.681
At n: 140 the rmse: 0.681
At n: 150 the rmse: 0.681
At n: 160 the rmse: 0.681
At n: 170 the rmse: 0.681
At n: 180 the rmse: 0.681
At n: 190 the rmse: 0.681


#### Question 4
###### Let's select the best max_depth:
###### Try different values of max_depth: [10, 15, 20, 25]
###### For each of these values,
###### try different values of n_estimators from 10 till 200 (with step 10)
###### calculate the mean RMSE
###### Fix the random seed: random_state=1
###### What's the best max_depth, using the mean RMSE?

In [75]:
for m in [10, 15, 20, 25]:
    for n in range(10, 210, 10):
        rmse = evaluate_rmse(n, m)
        print(f"At m: {m} the rmse: {rmse}") 

At m: 10 the rmse: 0.695
At m: 10 the rmse: 0.676
At m: 10 the rmse: 0.679
At m: 10 the rmse: 0.678
At m: 10 the rmse: 0.676
At m: 10 the rmse: 0.674
At m: 10 the rmse: 0.677
At m: 10 the rmse: 0.677
At m: 10 the rmse: 0.677
At m: 10 the rmse: 0.675
At m: 10 the rmse: 0.675
At m: 10 the rmse: 0.673
At m: 10 the rmse: 0.673
At m: 10 the rmse: 0.673
At m: 10 the rmse: 0.673
At m: 10 the rmse: 0.673
At m: 10 the rmse: 0.673
At m: 10 the rmse: 0.674
At m: 10 the rmse: 0.674
At m: 10 the rmse: 0.674
At m: 15 the rmse: 0.7
At m: 15 the rmse: 0.682
At m: 15 the rmse: 0.684
At m: 15 the rmse: 0.684
At m: 15 the rmse: 0.683
At m: 15 the rmse: 0.681
At m: 15 the rmse: 0.683
At m: 15 the rmse: 0.684
At m: 15 the rmse: 0.684
At m: 15 the rmse: 0.682
At m: 15 the rmse: 0.681
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.679
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.68
At m: 15 the rmse: 0.68
At m: 20 t

#### Question 5
###### We can extract feature importance information from tree-based models.

###### At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

###### In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

###### For this homework question, we'll find the most important feature:

###### Train the model with these parameters:
###### n_estimators=10,
###### max_depth=20,
###### random_state=1,
###### n_jobs=-1 (optional)
###### Get the feature importance information from this model
###### What's the most important feature (among these 4)? 

In [143]:
rfr= RandomForestRegressor(max_depth=20, n_estimators=10, random_state=1, n_jobs=-1)

In [146]:
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [174]:
feature_importance = rfr.feature_importances_

In [175]:
df_names = list(df_train.columns)
df_names

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'ocean_proximity']

In [179]:
for i,v in enumerate(feature_importance):
    print(f'For {i}, Score: {v}')

For 0, Score: 0.015514817118507777
For 1, Score: 0.030575615967311044
For 2, Score: 0.1014549933869993
For 3, Score: 0.0864817459789134
For 4, Score: 0.33158134816183
For 5, Score: 0.22276130307713812
For 6, Score: 0.14609650514660152
For 7, Score: 0.028100677252147738
For 8, Score: 0.015851085431664046
For 9, Score: 0.0215819084788871


In [180]:
df_names[4]

'total_bedrooms'

#### Question 6
###### Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

###### Install XGBoost
###### Create DMatrix for train and validation
###### Create a watchlist
###### Train a model with these parameters for 100 rounds:
###### xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
###### Now change eta from 0.3 to 0.1.

###### Which eta leads to the best RMSE score on the validation dataset?

In [52]:
import xgboost as xgb
from sklearn.metrics import precision_score

In [42]:
df_train['ocean_proximity'] = df_train['ocean_proximity'].str.lower().str.replace('<1h','1h').str.replace(' ', '_')

In [39]:
train_dummy = pd.get_dummies(df_train)

In [40]:
df_val['ocean_proximity'] = df_val['ocean_proximity'].str.lower().str.replace('<1h','1h').str.replace(' ', '_')
val_dummy = pd.get_dummies(df_val)

In [43]:
d_train = xgb.DMatrix(train_dummy, label=y_train)
d_val = xgb.DMatrix(val_dummy,label=y_test)

In [45]:
xgb_params = {
'eta': 0.3, 
'max_depth': 6,
'min_child_weight': 1,

'objective': 'reg:squarederror',
'nthread': 8,

'seed': 1,
'verbosity': 1,
}
num_round = 100  

In [46]:
bst = xgb.train(xgb_params, d_train, num_round)
preds = bst.predict(d_val)
preds

array([12.246578 , 12.132448 , 11.651083 , ..., 12.083826 , 11.0586195,
       11.775838 ], dtype=float32)

In [55]:
xgb_params = {
'eta': 0.1, 
'max_depth': 6,
'min_child_weight': 1,

'objective': 'reg:squarederror',
'nthread': 8,

'seed': 1,
'verbosity': 1,
}

In [56]:
bst_2 = xgb.train(xgb_params, d_train, num_round)
pred_2 = bst.predict(d_val)
pred_2

array([12.246578 , 12.132448 , 11.651083 , ..., 12.083826 , 11.0586195,
       11.775838 ], dtype=float32)

#### Ans: Same